In [1]:
from itertools import count
from collections import defaultdict

with open('inputs/24.txt') as file:
    input = file.read()

lines = input.split('\n')


In [2]:
%%time

def parse(lines):
    bugs = defaultdict(lambda:0)
    for y in range(5):
        for x in range(5):
            bugs[complex(x,y)] = 1 if lines[y][x]=='#' else 0
    return bugs

init = parse(lines)


Wall time: 0 ns


In [3]:

def pretty(step,bugs):
    print(step)
    for y in range(5):
        l = ''
        for x in range(5):
            l+='.#'[bugs[complex(x,y)]]
        print(l)

def iterate1(bugs):
    bugs = bugs.copy()
    nxt = defaultdict(lambda:0)
    visited = {repr(dict(bugs)):0}
    for step in count(1):
        for y in range(5):
            for x in range(5):
                p = complex(x,y)
                c = bugs[p]
                ns = bugs[p+1]+bugs[p-1]+bugs[p+1j]+bugs[p-1j]
                nxt[p] = int(ns==1 or ns==2 and not c)
        bugs,nxt = nxt,bugs
        s = repr(dict(bugs))
        if s in visited:
            break
        visited[s]=step

    biodiv = 0
    for y in range(5):
        for x in range(5):
            dx = 2**x
            dy = 32**y
            c = bugs[complex(x,y)]
            biodiv += c*dx*dy
    return biodiv


In [4]:
%%time

iterate1(init)


Wall time: 1.99 ms


17863741

In [5]:
def iterate2(bugs, steps):
    layers = defaultdict(lambda: defaultdict(lambda: 0))
    nxt = defaultdict(lambda: defaultdict(lambda: 0))
    layers[0] = bugs.copy()
    lmin,lmax = -1,1

    def get(l,p,d):
        x,y = int(p.real),int(p.imag)
        dx,dy = int(d.real),int(d.imag)
        nx,ny = x+dx,y+dy
        if nx<0:
            return layers[l-1][1+2j]
        if nx>4:
            return layers[l-1][3+2j]
        if ny<0:
            return layers[l-1][2+1j]
        if ny>4:
            return layers[l-1][2+3j]
        if nx==2 and ny==2:
            if dx==1:
                return sum([layers[l+1][complex(0,i)] for i in range(5)])
            if dx==-1:
                return sum([layers[l+1][complex(4,i)] for i in range(5)])
            if dy==1:
                return sum([layers[l+1][complex(i,0)] for i in range(5)])
            if dy==-1:
                return sum([layers[l+1][complex(i,4)] for i in range(5)])
            raise Exception('something wrong',l,p,d)
        return layers[l][p+d]
        
    for step in range(1,steps+1):
        for l in range(lmin,lmax+1):
            bugs = layers[l]
            for y in range(5):
                for x in range(5):
                    if x==2 and y==2: continue
                    p = complex(x,y)
                    c = bugs[p]
                    ns = get(l,p,1)+get(l,p,-1)+get(l,p,1j)+get(l,p,-1j)
                    nc = int(ns==1 or ns==2 and not c)
                    nxt[l][p] = nc
                    if nc:
                        lmin = min(lmin,l-1)
                        lmax = max(lmax,l+1)
        nxt,layers = layers,nxt

    total = 0
    for l in range(lmin,lmax+1):
        for y in range(5):
            for x in range(5):
                total += layers[l][complex(x,y)]
    return total


In [6]:
%%time

iterate2(init,200)


Wall time: 2.53 s


2029